<a href="https://colab.research.google.com/github/shraddha-an/cnns/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Classification with CNN - Cats vs Owls

# 1) Introduction


---



#### **Skills Demonstrated**: Data Collection, Image Augmentation, Deep Learning, Image Classification, Web Scraping.


In this Deep Learning Project, I built a CNN to classify images of cats and owls.


I wrote a Python script to download photos of cats and owls from Flickr using the [flickrapi](https://github.com/sybrenstuvel/flickrapi) and the urllib library.






# 2) Data Collection


---



To use the Flickr API service, you'll have to apply for an API Authentication key [here](https://www.flickr.com/services/api/misc.api_keys.html).



1.  Download the flickrapi library
    ```
    pip install flickrapi
    ```

2.  Download the urllib library
    ```
    pip install urllib
    ```





### a) Importing libraries

In [ ]:
import urllib
import flickrapi

from time import sleep

### b) API Authentication

In [ ]:
# To authenticate, plug in your API Key as the first argument and the Secret Code as the second.
flickr = flickrapi.FlickrAPI('API_Key', 'Secret_code', cache = True)

# Getting the photos through the walk method
cat_photos = flickr.walk(text = 'cat',
                     tag_mode = 'all',
                     tags = 'cat',
                     extras = 'url_c',
                     per_page = 1000,
                     sort = 'relevance')

owl_photos = flickr.walk(text = 'owl',
                     tag_mode = 'all',
                     tags = 'owl',
                     extras = 'url_c',
                     per_page = 1000,
                     sort = 'relevance')

### c) Downloading Images
Create a train and a test folder. Inside each of the 2 folders, create 2 more folders and name them cat and owl.

Make sure to download the cat images into the cat folder and the owl images into the owl folder.

After the images have been downloaded, move 10 images each of cats & owls from the train folder to the test folder.

In [ ]:
# Extracting URLs of photos from the response
cat_urls = []
for i, photo in enumerate(cat_photos):

    cat_urls.append(photo.get('url_c'))

    # get 1000 urls
    if i > 1050:
        break

owl_urls = []
for i, photo in enumerate(owl_photos):

    owl_urls.append(photo.get('url_c'))

    # get 1000 urls
    if i > 1050:
        break


# Removing None type objects & taking only the first 960 URLs. 950 images for training & 10 images for testing
cat_urls = [x for x in cat_urls if x is not None][:960]
owl_urls = [x for x in owl_urls if x is not None][:960]

# =====================================  Downloading the images to folders ==========================================
# 1) Cat Images
for count, url in enumerate(cat_urls[:480]):
    urllib.request.urlretrieve(url, 'training_folder_path/cat'+ str(count) + '.jpg')

sleep(100)

for count, url in enumerate(cat_urls[480:]):
    urllib.request.urlretrieve(url, 'training_folder_path/cat'+ str(480 + count) + '.jpg')

# Owl Images
for count, url in enumerate(owl_urls[:480]):
    urllib.request.urlretrieve(url, 'training_folder_path/owl'+ str(count) + '.jpg')

sleep(100)

for count, url in enumerate(owl_urls[480: ]):
    urllib.request.urlretrieve(url, 'training_folder_path/owl'+ str(450 + count) + '.jpg')


# 3) CNN Architecture

In [ ]:
# Import the Sequential model and layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

batch_size = 5

# 4) Image Augmentation

In [ ]:
# Training Augmentation configuration
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Testing Augmentation - Only Rescaling
test_datagen = ImageDataGenerator(rescale = 1./255)

# Generates batches of Augmented Image data
train_generator = train_datagen.flow_from_directory('train/', target_size = (300, 300), 
                                                    batch_size = batch_size,
                                                    class_mode = 'binary') 

# Generator for validation data
validation_generator = test_datagen.flow_from_directory('test/', 
                                                        target_size = (300, 300),
                                                        batch_size = batch_size,
                                                        class_mode = 'binary')

# 5) Training & Model Evaluation

In [ ]:
model.fit_generator(train_generator,
                    epochs = 5,
                    validation_data = validation_generator,
                    verbose = 1)

# Evaluating model performance on Testing data
loss, accuracy = model.evaluate(validation_generator)

print('\nAccuracy: ', accuracy, '\nLoss: ', loss)